## Load library

In [9]:
import pandas as pd
import numpy as np
import anndata as ad
import scanpy as sc

import warnings
warnings.filterwarnings("ignore")

In [35]:
samples = pd.read_csv("sample_name.txt", header=None)[0].tolist()

metadata_sample_path = "sample_metadata.csv"

adata_count = None
adata_tpm = None

for sample in samples:
    transcript = pd.read_csv(f'output_{sample}/transcripts.txt', header=None, index_col=0)
    transcript.index.names = ['']
    counts = sc.read_mtx(f'output_{sample}/matrix.abundance.mtx', dtype='float32')
    counts.obs.index = [sample]
    counts.var.index = transcript.index.tolist()
    
    tpm = sc.read_mtx(f'output_{sample}/matrix.abundance.tpm.mtx', dtype='float32')
    tpm.obs.index = [sample]
    tpm.var.index = transcript.index.tolist()

    if adata_count is None:
        adata_count = counts.copy()
        adata_tpm = tpm.copy()
        continue

    adata_count = ad.concat([adata_count, counts], axis=0)
    adata_count.var = pd.concat([adata_count.var, transcript], axis=1).loc[adata_count.var.index, :]

    adata_tpm = ad.concat([adata_tpm, tpm], axis=0)
    adata_tpm.var = pd.concat([adata_tpm.var, transcript], axis=1).loc[adata_tpm.var.index, :]

metadata_sample = pd.read_csv(metadata_sample_path)
metadata_sample.index = metadata_sample['Sample Name'].values

adata_count.obs = pd.concat([adata_count.obs, metadata_sample], axis=1)
adata_tpm.obs = pd.concat([adata_tpm.obs, metadata_sample], axis=1)


adata_count.write(filename="transcript_exp_count.h5ad")
adata_tpm.write(filename="transcript_exp_tpm.h5ad")